#Mount del drive e installazione/importazione librerie necessarie

In [ ]:
!pip install torch torchvision
!pip install rasterio
!pip install geopandas
!pip install matplotlib
!pip install albumentations
!pip install tqdm


In [24]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
percorso =  "/content/drive/MyDrive/Progetto_laboratorio"
import sys
sys.path.append(percorso)
import fiona
import os
from torch.utils.data import DataLoader, WeightedRandomSampler

import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF


Mounted at /content/drive


In [26]:
import importlib
import dataset
importlib.reload(dataset)
import model_unet
importlib.reload(model_unet)
import utils
importlib.reload(utils)
from dataset import Dataset
from model_unet import UNET
from utils import (
    save_checkpoint,
    load_checkpoint,
    check_precision,
    check_recall,
    check_f1,
    check_accuracy,
    check_metrics,
    calculate_metrics,
)

In [20]:
#Hyperparameters
LEARNING_RATE = [1e-3,1e-4,1e-5]
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 15
NUM_EPOCHS = 5
NUM_WORKERS = 2
#IMAGE_HEIGHT = 256
#IMAGE_WIDTH = 256
PIN_MEMORY = True
LOAD_MODEL = False
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = percorso + '/Dataset/train'
VAL_DIR = percorso + "/Dataset/validation"
TEST_DIR = percorso + "/Dataset/test"



In [5]:
import random
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False


#Training modello

In [6]:
def train_fn(loader_no_aug,loader_aug, model, optimizer, loss_fn, scaler,use_augmentation):
  train_loss = 0.0
  model.train()                                                    #imposta il modello in training mode
  loader = loader_aug if use_augmentation else loader_no_aug
  loop = tqdm(loader)                                       #progress bar

  for batch_idx, (data, targets) in enumerate(loop):                       #iteriamo sui batch forniti dal loader
    if (data.shape[1] != 6 and data.shape[3] == 6):
      data = data.permute(0,3,1,2)
    if (data.shape[1] != 6 and data.shape[2] != 6):
      data = data.permute(0,2,1,3)
    data = data.to(device = DEVICE)                                      #sposta i dati sul device , CPU o GPU
    targets = targets.float().unsqueeze(1).to(device = DEVICE)             #convertiamo targets in float e aggiungiamo una dimensione extra
    #print(data.shape)
    #print(targets.shape)
    # forward
    with torch.cuda.amp.autocast():                                     #abilita il mixed precision training
      predictions = model(data)                                          #calcoliamo predizioni
      loss = loss_fn(predictions, targets)                                 #calcoliamo la loss

    # backward
    optimizer.zero_grad()                                                  #azzera i gradienti dell'optimizer
    scaler.scale(loss).backward()                                          #scala la loss e calcola i gradienti con backward
    scaler.step(optimizer)                                                 #aggiorna i parametri
    scaler.update()                                                        #aggiorna lo scaler

    train_loss+=loss.item()

    # update tqdm loop
    loop.set_postfix(loss = loss.item())                                     #aggiorna il progress bar con il valore della loss

  train_loss/=len(loader)
  return train_loss



In [9]:

train_transform_aug = A.Compose([
      A.Rotate(limit=35, p=1.0),
      A.HorizontalFlip(p=0.5),
      A.VerticalFlip(p=0.1),
      A.Normalize(
          mean=[0.0, 0.0, 0.0],
          std=[1.0, 1.0, 1.0],
          max_pixel_value=255.0,
      ),
      ToTensorV2(),
  ], is_check_shapes=False)

train_transform_no_aug = A.Compose([
      A.Normalize(
          mean=[0.0, 0.0, 0.0],
          std=[1.0, 1.0, 1.0],
          max_pixel_value=255.0,
      ),
      ToTensorV2(),
  ], is_check_shapes=False)

val_transform = A.Compose([
    A.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=[1.0, 1.0, 1.0],
        max_pixel_value=255.0,
    ),
    ToTensorV2(),
], is_check_shapes=False)


In [10]:
#bilanciamento dataset con weightd random sampler, assegna pesi diversi ad ogni campione in base alla frequenza delle classi

train_ds = dataset.Dataset(root = percorso + "/Dataset/train",transform = train_transform_aug)
val_ds = dataset.Dataset(root = percorso + "/Dataset/validation" , transform = val_transform)

#Calcola le occorrenze di ogni classe
class_counts = torch.zeros(2)
mask_list = []
for i in range(len(train_ds)):
  _, mask = train_ds[i]
  mask_list.append(mask)
  mask_tensor = torch.from_numpy(mask)
  class_counts += torch.bincount(mask_tensor.flatten(), minlength=2)



In [11]:
weights = 1. / class_counts.float()
weights[weights == 0] = 1e-6
weights = weights / weights.sum()

  # Sostituisci i valori negativi con un piccolo valore positivo

# Crea samples_weights
samples_weights = torch.zeros(len(train_ds), dtype=torch.float)
for i in range(len(train_ds)):
    #_, mask = train_ds[i]
    mask_tensor = torch.from_numpy(mask_list[i])
    class_weight = weights[mask_tensor.flatten().unique()]
    samples_weights[i] = class_weight.mean()
# Crea un sampler


if torch.any(samples_weights <= 0):
    print("Ci sono valori negativi in samples_weights, li correggo...")
    samples_weights[samples_weights <= 0] = 1e-6
# Verifica che non ci siano valori negativi o NaN

sampler = WeightedRandomSampler(weights=samples_weights, num_samples=len(train_ds), replacement=True)            #replacement=true permette di scegliere piu volte lo stesso campione e permette l'oversamppling per sovracampionarie le classi minoritarie

# Crea un DataLoader con il sampler
balanced_train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=4, pin_memory=PIN_MEMORY)



[17, 23, 29, 38, 35, 38, 35, 20, 35, 34, 10, 18, 27, 9, 3, 9, 34, 10, 23, 15, 10, 32, 14, 47, 45, 30, 36, 44, 17, 7, 20, 12, 23, 7, 15, 28, 0, 17, 10, 47, 8, 22, 37, 13, 17, 0, 3, 23]


<ipython-input-11-85a70f3a3d6a>:15: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  class_weight = weights[mask_tensor.flatten().unique()]


#Prova del modello

In [28]:
def main():

  train_ds_aug = dataset.Dataset(root = percorso + "/Dataset/train",transform = train_transform_aug)
  train_ds_no_aug = dataset.Dataset(root = percorso + "/Dataset/train",transform = train_transform_no_aug)
  val_ds = dataset.Dataset(root = percorso + "/Dataset/validation" , transform = val_transform)

  train_loader_aug = DataLoader(train_ds_aug, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=PIN_MEMORY)
  train_loader_no_aug = DataLoader(train_ds_no_aug, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=PIN_MEMORY)
  val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=PIN_MEMORY)
  balanced_train_loader_aug = DataLoader(train_ds_aug, batch_size=BATCH_SIZE, sampler=sampler, num_workers=4, pin_memory=PIN_MEMORY)
  balanced_train_loader_no_aug = DataLoader(train_ds_no_aug, batch_size=BATCH_SIZE, sampler=sampler, num_workers=4, pin_memory=PIN_MEMORY)

  class_weights = weights[1] / weights[0]                 #per bilanciare le classi con 1 classe minoritaria e 0 classe maggioritaria


  model = UNET(in_channels=6, out_channels=1).to(device = DEVICE)
  loss_fn = nn.BCEWithLogitsLoss(pos_weight = class_weights)
  for lr in LEARNING_RATE:
    print('Proviamo lr : ',lr)
    optimizer = optim.Adam(model.parameters(), lr, weight_decay=1e-4)         #aiuta a prevenire che i pesi diventino troppo grandi
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min",factor=0.001,patience=3)                 #riduce il lr di 0.1 ad intervalli regolari
    scaler = torch.cuda.amp.GradScaler()
    set_seed(42)

    train_losses=[]
    val_losses=[]
    val_precisions=[]
    val_recalls=[]
    val_f1s=[]
    val_accuracies=[]

    scaler = torch.cuda.amp.GradScaler()

    if LOAD_MODEL:
      load_checkpoint(torch.load("model.pth.tar"), model)

    all_predictions = []
    all_targets = []
    print("Training")

    for epoch in range (NUM_EPOCHS):
      use_augmentation = (epoch % 2 == 0)
      train_loss=train_fn(loader_no_aug=balanced_train_loader_no_aug,loader_aug=balanced_train_loader_aug, model=model, optimizer=optimizer, loss_fn=loss_fn, scaler=scaler,use_augmentation=use_augmentation)

      train_losses.append(train_loss)

      val_loss= 0.0
      with torch.no_grad():
        for images,labels in val_loader:
          images = images.to(DEVICE)
          labels = labels.float().unsqueeze(1).to(DEVICE)
          outputs = model(images)
          loss = loss_fn(outputs,labels)
          val_loss+=loss.item()

        val_loss/=len(val_loader)
        val_losses.append(val_loss)

      accuracy,precision,recall,f1 = calculate_metrics(val_loader,model,DEVICE)
      val_precisions.append(precision)
      val_recalls.append(recall)
      val_f1s.append(f1)
      val_accuracies.append(accuracy)

      print('train loss: ',train_loss)
      print('val_loss : ',val_loss)
      print('accuracy : ',accuracy*100)
      print('precision : ',precision)
      print('recall : ', recall)
      print('f1 : ', f1)

      scheduler.step(val_loss)



In [29]:

if __name__ == "__main__":
  try:
      torch.multiprocessing.set_start_method('spawn', force=True)
  except RuntimeError:
      pass
  torch.cuda.empty_cache()
  main()


Proviamo lr :  0.001
Training


100%|██████████| 4/4 [01:44<00:00, 26.13s/it, loss=1.14]


tensor([[[[0.5076, 0.5095, 0.5080,  ..., 0.5115, 0.5099, 0.5084],
          [0.5056, 0.5064, 0.5048,  ..., 0.5107, 0.5110, 0.5078],
          [0.5048, 0.5072, 0.5022,  ..., 0.5124, 0.5107, 0.5091],
          ...,
          [0.5090, 0.5126, 0.5123,  ..., 0.5152, 0.5161, 0.5113],
          [0.5087, 0.5143, 0.5131,  ..., 0.5168, 0.5159, 0.5127],
          [0.5079, 0.5095, 0.5089,  ..., 0.5115, 0.5121, 0.5090]]],


        [[[0.5096, 0.5122, 0.5121,  ..., 0.5106, 0.5094, 0.5082],
          [0.5106, 0.5130, 0.5152,  ..., 0.5084, 0.5099, 0.5076],
          [0.5103, 0.5158, 0.5150,  ..., 0.5092, 0.5093, 0.5089],
          ...,
          [0.5127, 0.5171, 0.5186,  ..., 0.5184, 0.5182, 0.5127],
          [0.5113, 0.5181, 0.5169,  ..., 0.5197, 0.5182, 0.5138],
          [0.5099, 0.5127, 0.5125,  ..., 0.5145, 0.5143, 0.5104]]],


        [[[0.5070, 0.5083, 0.5069,  ..., 0.5065, 0.5058, 0.5059],
          [0.5040, 0.5039, 0.5016,  ..., 0.5002, 0.5021, 0.5029],
          [0.5030, 0.5037, 0.4983,  ..

100%|██████████| 4/4 [01:35<00:00, 23.80s/it, loss=0.674]


tensor([[[[0.4839, 0.4825, 0.4794,  ..., 0.4831, 0.4794, 0.4776],
          [0.4795, 0.4739, 0.4700,  ..., 0.4801, 0.4787, 0.4763],
          [0.4823, 0.4812, 0.4751,  ..., 0.4873, 0.4830, 0.4814],
          ...,
          [0.4845, 0.4861, 0.4892,  ..., 0.4954, 0.4972, 0.4867],
          [0.4848, 0.4959, 0.4915,  ..., 0.5035, 0.4991, 0.4892],
          [0.4845, 0.4865, 0.4888,  ..., 0.4914, 0.4930, 0.4862]]],


        [[[0.4831, 0.4829, 0.4804,  ..., 0.4823, 0.4790, 0.4773],
          [0.4810, 0.4823, 0.4842,  ..., 0.4771, 0.4782, 0.4758],
          [0.4834, 0.4908, 0.4860,  ..., 0.4848, 0.4828, 0.4820],
          ...,
          [0.4883, 0.5014, 0.5027,  ..., 0.5053, 0.5030, 0.4936],
          [0.4893, 0.5055, 0.5002,  ..., 0.5121, 0.5052, 0.4944],
          [0.4877, 0.4923, 0.4955,  ..., 0.4971, 0.4970, 0.4896]]],


        [[[0.4847, 0.4840, 0.4815,  ..., 0.4812, 0.4774, 0.4753],
          [0.4791, 0.4719, 0.4673,  ..., 0.4670, 0.4650, 0.4673],
          [0.4821, 0.4780, 0.4709,  ..

...

tensor([[[[0.4366, 0.4233, 0.4161,  ..., 0.4674, 0.4553, 0.4371],
          [0.4208, 0.4140, 0.4061,  ..., 0.4582, 0.4437, 0.4484],
          [0.4203, 0.4088, 0.3975,  ..., 0.4720, 0.4522, 0.4551],
          ...,
          [0.4561, 0.4701, 0.4655,  ..., 0.4397, 0.4466, 0.4756],
          [0.4573, 0.4725, 0.4667,  ..., 0.4495, 0.4547, 0.4789],
          [0.4586, 0.4679, 0.4650,  ..., 0.4698, 0.4657, 0.4622]]],


        [[[0.4626, 0.4708, 0.4712,  ..., 0.4425, 0.4434, 0.4322],
          [0.4501, 0.4595, 0.4590,  ..., 0.4388, 0.4335, 0.4390],
          [0.4542, 0.4664, 0.4609,  ..., 0.4506, 0.4447, 0.4468],
          ...,
          [0.4450, 0.4232, 0.4227,  ..., 0.3997, 0.4171, 0.4814],
          [0.4521, 0.4380, 0.4397,  ..., 0.4140, 0.4334, 0.4887],
          [0.4624, 0.4695, 0.4712,  ..., 0.4712, 0.4733, 0.4763]]],


        [[[0.4208, 0.4060, 0.4004,  ..., 0.3935, 0.4060, 0.3997],
          [0.3975, 0.3830, 0.3604,  ..., 0.3576, 0.3708, 0.3849],
          [0.3946, 0.3698, 0.3491,  ..

100%|██████████| 4/4 [01:34<00:00, 23.55s/it, loss=0.58]


tensor([[[[0.4291, 0.4134, 0.4055,  ..., 0.4706, 0.4593, 0.4357],
          [0.4097, 0.4051, 0.3922,  ..., 0.4614, 0.4477, 0.4512],
          [0.4094, 0.3959, 0.3829,  ..., 0.4786, 0.4584, 0.4595],
          ...,
          [0.4568, 0.4706, 0.4677,  ..., 0.4493, 0.4568, 0.4838],
          [0.4586, 0.4750, 0.4684,  ..., 0.4581, 0.4661, 0.4870],
          [0.4586, 0.4736, 0.4694,  ..., 0.4771, 0.4736, 0.4667]]],


        [[[0.4640, 0.4706, 0.4728,  ..., 0.4423, 0.4433, 0.4297],
          [0.4501, 0.4638, 0.4617,  ..., 0.4392, 0.4356, 0.4407],
          [0.4544, 0.4722, 0.4675,  ..., 0.4523, 0.4487, 0.4487],
          ...,
          [0.4502, 0.4320, 0.4291,  ..., 0.4130, 0.4270, 0.4906],
          [0.4582, 0.4444, 0.4451,  ..., 0.4252, 0.4447, 0.4983],
          [0.4666, 0.4786, 0.4802,  ..., 0.4809, 0.4827, 0.4847]]],


        [[[0.4123, 0.3960, 0.3889,  ..., 0.3808, 0.3939, 0.3914],
          [0.3873, 0.3694, 0.3427,  ..., 0.3399, 0.3527, 0.3731],
          [0.3820, 0.3537, 0.3302,  ..

  0%|          | 0/4 [00:20<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label='Accuracy')
plt.plot(val_precisions, label='Precision')
plt.plot(val_recalls, label='Recall')
plt.plot(val_f1s, label='F1-Score')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.legend()
plt.title('Validation Metrics')

plt.tight_layout()
plt.savefig('training_metrics.png')
plt.show()
